In [33]:
# statistical libraries
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx

# visualization libraries
import matplotlib.pyplot as plt
from matplotlib import cm
import plotly
import plotly.plotly as py
from plotly.graph_objs import Scatter, Layout

# import own library functionality
import paths
from TRMM import TRMM
from Visualization import Visualization

# force autoreload of external modules on save
%load_ext autoreload
%autoreload 2
%matplotlib inline
plotly.offline.init_notebook_mode(connected=True)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
YEARS = range(1998, 2017)
MONTHS = [3, 4, 5]
AGGREGATION_RESOLUTION = 6.125

In [35]:
df = TRMM.load_dataset(
    [1998, 1999],
    [3],
    aggregation_resolution=AGGREGATION_RESOLUTION,
    timestamp=True,
    invalidate=False,
    lon_slice=slice(61.125, 97.625),
    lat_slice=slice(4.125, 40.625),
    version='v3')

df

> Loading from cache...


888710400    888796800    888883200    888969600  \
latitude longitude                                                      
7.0625   64.0625     17.519999    37.979999     0.000000     3.120000   
         70.1875     62.069998   132.749995    60.899998   309.239991   
         76.3125     18.766308   406.731051   381.768621   305.470296   
         82.4375     10.965764  1584.294771  1070.678084   540.545089   
         88.5625      0.030000  2192.699962   989.519978   349.979993   
         94.6875     40.127661   432.045043   765.144575   365.105595   
13.1875  64.0625     53.459999     0.030000     0.000000     0.030000   
         70.1875      0.030000     0.000000     0.090000     0.150000   
         76.3125      6.779792    11.418573     6.863090     5.550914   
         82.4375      0.000000     0.000000     0.210000    43.475724   
         88.5625      0.060000     0.000000     0.060000     0.030000   
         94.6875      2.692171     3.416679    32.587729    19.256488   
19.3125  64.0625      0.000000     0.060000     1.710000    39.150000   
         70.1875      0.300173     0.252000     2.180408     0.657044   
         76.3125     12.106768     7.477048    19.500555   174.720077   
         82.4375    505.504714    48.904089    42.133478    98.674865   
         88.5625    707.671285   151.728531     5.763819     1.163081   
         94.6875      2.797981     2.059666     0.353451     0.771712   
25.4375  64.0625     89.195736  1863.004745  1286.814065    11.901707   
         70.1875      0.684000   366.080278   362.871490    57.088668   
         76.3125     14.401451    49.213390   306.389456  1048.450592   
         82.4375      3.869597     1.639209    11.477815   348.505099   
         88.5625     18.154110     2.120412    27.867544    52.484732   
         94.6875    215.293199   496.649836    19.387570   132.508040   
31.5625  64.0625    835.956547   299.252000    75.541742    56.613432   
         70.1875    192.653844  1115.460786   945.145320    15.956720   
         76.3125     49.237290  2332.359007  4244.255697  1324.095406   
         82.4375      8.522765    18.125187  1361.261074   333.893522   
         88.5625     58.047924    21.183042   677.705247   457.441474   
         94.6875    393.504266   237.178185   107.247511  1706.765943   
37.6875  64.0625    377.488272   276.992203    13.907165    11.136185   
         70.1875    458.772750   575.980570   205.626405     0.000000   
         76.3125     21.042933   418.478688   241.418161    46.118961   
         82.4375      0.000000    61.117254    25.284489    31.475115   
         88.5625      8.065126     0.000000   144.385255   121.440225   
         94.6875     21.843227     0.796813   243.452351    16.284473   

                      889056000   889142400    889228800   889315200  \
latitude longitude                                                     
7.0625   64.0625      13.770000   40.710000     2.430000  126.989996   
         70.1875      59.849999  163.229994     0.090000   32.579999   
         76.3125     129.686964   96.375979     5.896414   14.139805   
         82.4375     602.719107  965.044956    77.829585  327.843446   
         88.5625      41.939999    6.960000     6.900000   10.440000   
         94.6875     221.634823   77.496575    65.745736  184.723788   
13.1875  64.0625       0.090000    0.000000     0.060000    0.270000   
         70.1875       0.162000    0.000000     0.030000    0.006000   
         76.3125       4.350656   13.864870     0.745503    2.803146   
         82.4375       0.000000    0.000000     1.452584    1.002570   
         88.5625       0.000000    0.030000     0.180000    0.090000   
         94.6875       1.146000    0.784866     0.210000    0.210000   
19.3125  64.0625       0.030000    0.030000     0.030000    3.330000   
         70.1875       0.000000    0.000000     0.210000    2.514000   
         76.3125      44.544365    2.380269     0.584704    0.421006   
         82.4375      

In [84]:
def calculate_time_lag(i, j):
        """
        Calculate the time lag between two three-tuples

        :param i: Tuple like (l-1, l, l+1)
        :param j: Tuple like (m-1, m, m+1)

        :return: Time lag (number)
        """
        
        return 0.5 * min(i[2] - i[1], i[1] - i[0], j[2] - j[1], j[1] - j[0])

In [100]:
def calculate_synchronization(row1, row2):
        """
        Calculate the number of synchronous events between two rows

        :param row1: First row
        :param row2: Second row

        :return: The number of synchronous events (integer)
        """

        # initialize the total number of synchronous events for the two grid points
        num_sync_events = 0

        # iterate over all windows of size three in the first row
        for i_prev, i_current, i_next in TRMM.sliding_window(row1, window_size=3, padded=True):
            # calculate the max timestamp of row2 that could be synchronous
            earliest = i_current - 0.5 * min(i_current - i_prev, i_next - i_current)

            # print(row2.index)
            before_earliest = row2.index.get_loc(earliest, method='nearest') - 1
            after_current = row2.index.get_loc(i_next, method='nearest') + 1
            
            if before_earliest < 0:
                before_earliest = 0
            if after_current > len(row2.index) - 1:
                after_current = len(row2.index) - 1
                
            print(before_earliest, row2.iloc[before_earliest], after_current, row2.iloc[after_current])

            # pass through all events at the same or at a later time at the second location
            # for j_prev, j_current, j_next in TRMM.sliding_window(row2, window_size=3):
            for j_prev, j_current, j_next in TRMM.sliding_window(row2, window_size=3, padded=True):
                # the following optimizations reduce runtime from ~0.3s to 0.02s

                # calculate the difference
                current_diff = i_current - j_current
                # print('current diff', current_diff)

                # if the difference gets negative, break
                # the second pass will encompass these combinations
                # if current_diff < 0:
                    # break

                # if the events occur at the same time, they will be counted twice
                # thus only add half the value
                if current_diff == 0:
                    num_sync_events += 0.5
                    continue

                # continue for timestamps that cannot possibly be synchronous
                # i.e. are much too early
                # if j_current < earliest:
                    # continue

                # calculate the time lag based on the current state of the two sliding windows
                time_lag = TRMM.calculate_time_lag((i_prev, i_current, i_next), (j_prev, j_current, j_next))
                # print('time lag', i_current, j_current, time_lag)
                
                # if the second event lies within the time lag, it is fully synchronous
                if 0 < current_diff <= time_lag:
                    num_sync_events += 1.0
                    
        return num_sync_events

In [90]:
df.iloc[0].quantile(0.9)

1671.3299574583768

In [40]:
df.iloc[0][df.iloc[0] >= df.iloc[0].quantile(0.9)]

890179200    1677.089958
890265600    5501.519918
890438400    1723.919950
891043200    3226.709923
891129600    4252.499904
921715200    2439.539940
921801600    3157.889906
Name: (7.0625, 64.0625), dtype: float64

In [65]:
extreme_events = TRMM.extract_extreme_events(df, quantile=0.90, pad=False)
extreme_events

888710400  888796800  888883200  888969600  889056000  \
latitude longitude                                                          
7.0625   64.0625        False      False      False      False      False   
         70.1875        False      False      False      False      False   
         76.3125        False      False      False      False      False   
         82.4375        False      False      False      False      False   
         88.5625        False       True      False      False      False   
         94.6875        False      False      False      False      False   
13.1875  64.0625         True      False      False      False      False   
         70.1875        False      False      False      False      False   
         76.3125        False      False      False      False      False   
         82.4375        False      False      False      False      False   
         88.5625        False      False      False      False      False   
         94.6875        False      False      False      False      False   
19.3125  64.0625        False      False      False      False      False   
         70.1875        False      False      False      False      False   
         76.3125        False      False      False       True      False   
         82.4375         True      False      False      False      False   
         88.5625        False      False      False      False      False   
         94.6875        False      False      False      False      False   
25.4375  64.0625        False       True       True      False      False   
         70.1875        False       True       True      False      False   
         76.3125        False      False       True       True      False   
         82.4375        False      False      False      False       True   
         88.5625        False      False      False      False      False   
         94.6875        False      False      False      False      False   
31.5625  64.0625        False      False      False      False      False   
         70.1875        False      False      False      False      False   
         76.3125        False       True       True      False      False   
         82.4375        False      False       True      False      False   
         88.5625        False      False       True       True      False   
         94.6875        False      False      False       True       True   
37.6875  64.0625        False      False      False      False      False   
         70.1875        False      False      False      False      False   
         76.3125        False      False      False      False      False   
         82.4375        False      False      False      False      False   
         88.5625        False      False      False      False      False   
         94.6875        False      False      False      False      False   

                    889142400  889228800  889315200  889401600  889488000  \
latitude longitude                                                          
7.0625   64.0625        False      False      False      False      False   
         70.1875        False      False      False      False      False   
         76.3125        False      False      False      False      False   
         82.4375        False      False      False      False      False   
         88.5625        False      False      False      False      False   
         94.6875        False      False      False      False      False   
13.1875  64.0625        False      False      False      False      False   
         70.1875        False      False      False      False      False   
         76.3125        False      False      False      False      False   
         82.4375        False      False      False      False      False   
         88.5625        False      False      False      False      False   
         94.6875        False      False      False      False      False   
19.3125  64.062

In [66]:
row_0 = extreme_events.iloc[0]
row_1 = extreme_events.iloc[1]
row_10 = extreme_events.iloc[10]
row_30 = extreme_events.iloc[30]

In [67]:
true_0 = row_0[row_0]
true_0

890179200    True
890265600    True
890438400    True
891043200    True
891129600    True
921715200    True
921801600    True
Name: (7.0625, 64.0625), dtype: bool

In [68]:
true_1 = row_1[row_1]
true_1

890006400    True
890092800    True
890179200    True
890438400    True
891129600    True
921110400    True
921628800    True
Name: (7.0625, 70.1875), dtype: bool

In [69]:
true_10 = row_10[row_10]
true_10

889747200    True
889833600    True
890265600    True
890352000    True
921456000    True
921542400    True
922406400    True
Name: (13.1875, 88.5625), dtype: bool

In [70]:
true_30 = row_30[row_30]
true_30

890179200    True
891216000    True
920419200    True
920592000    True
922406400    True
922579200    True
922665600    True
Name: (37.6875, 64.0625), dtype: bool

In [101]:
true_t1 = pd.Series([True, True, True, True, True, True], index=[0, 7, 9, 10, 12, 999999999])
true_t2 = pd.Series([True, True, True, True, True, True], index=[0, 1, 5, 7, 10, 999999999])

calculate_synchronization(true_t1, true_t1)

0 True 2 True
0 True 3 True
1 True 4 True
2 True 5 True
3 True 5 True
4 True 5 True


3.0

In [102]:
true_t1 = pd.Series([True, True, True, True, True, True], index=[0, 7, 9, 10, 12, 999999999])
true_t2 = pd.Series([True, True, True, True, True, True], index=[0, 1, 5, 7, 10, 999999999])

calculate_synchronization(true_t1, true_t1)

0 True 2 True
0 True 3 True
1 True 4 True
2 True 5 True
3 True 5 True
4 True 5 True


3.0

In [104]:
for row1, row2 in [(true_0, true_0), (true_0, true_1), (true_0, true_10), (true_0, true_30), (true_t1, true_t2)]:
    sync1 = calculate_synchronization(row1, row2)
    sync2 = calculate_synchronization(row2, row1)    
    sync, count = TRMM.calculate_sync_strength(row1, row2)
    print(float(sync1), float(sync2), sync, float(count))

0 True 2 True
0 True 3 True
1 True 4 True
2 True 5 True
3 True 6 True
4 True 6 True
5 True 6 True
0 True 2 True
0 True 3 True
1 True 4 True
2 True 5 True
3 True 6 True
4 True 6 True
5 True 6 True
3.5 3.5 1.0 5.0
1 True 3 True
1 True 4 True
2 True 5 True
3 True 5 True
3 True 6 True
5 True 6 True
5 True 6 True
0 True 1 True
0 True 1 True
0 True 3 True
0 True 5 True
2 True 6 True
4 True 6 True
4 True 6 True
1.5 1.5 0.4 2.0
1 True 3 True
1 True 4 True
2 True 4 True
2 True 4 True
2 True 6 True
4 True 6 True
4 True 6 True
0 True 1 True
0 True 2 True
0 True 3 True
0 True 6 True
4 True 6 True
4 True 6 True
5 True 6 True
0.5 0.5 0.2 1.0
0 True 1 True
0 True 1 True
0 True 2 True
0 True 2 True
0 True 5 True
3 True 5 True
3 True 6 True
0 True 5 True
1 True 6 True
4 True 6 True
4 True 6 True
5 True 6 True
5 True 6 True
5 True 6 True
0.5 0.5 0.0 0.0
0 True 4 True
2 True 5 True
3 True 5 True
3 True 5 True
3 True 5 True
4 True 5 True
0 True 1 True
0 True 2 True
0 True 2 True
0 True 4 True
1 True 5 Tru

In [123]:
for row1, row2 in [(true_0, true_0), (true_0, true_1), (true_0, true_10), (true_0, true_30), (true_t1, true_t2)]:
    sync1 = calculate_synchronization(row1, row2)
    sync2 = calculate_synchronization(row2, row1)    
    sync, count = TRMM.calculate_sync_strength(row1, row2)
    print(float(sync1), float(sync2), sync, float(count))

3.5 3.5 1.0 7.0
1.5 1.5 0.42857142857142855 3.0
0.5 0.5 0.14285714285714285 1.0
0.5 0.5 0.14285714285714285 1.0
1.0 1.0 0.5 2.0


In [16]:
extreme_events.shape[0] * extreme_events.shape[1]

2232

In [12]:
extreme_events.sum().sum()

252

In [63]:
TRMM.calculate_synchronization(first_true, second_true)

1.0

In [64]:
TRMM.calculate_synchronization(second_true, first_true)

1.0

In [58]:
TRMM.calculate_synchronization(extreme_events.iloc[0], extreme_events.iloc[30])

30.0

In [41]:
sync_strength, sync_count = TRMM.calculate_sync_strength(extreme_events.iloc[0], extreme_events.iloc[1])

print(sync_strength, sync_count)

0.4 2.0


0.4